# 기본 태양광 데이터 

In [1]:
import pandas as pd
import numpy as np
import os
import glob
import random

import warnings
warnings.filterwarnings("ignore")

#pytorch 라이브러리 임포트
import torch
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

#scikit-learn 라이브러리
from sklearn.datasets import load_digits
from sklearn import datasets, model_selection

#pandas 라이브러리 임포트
import pandas as pd

#matplotlib 라이브러리 임포트
from matplotlib import pyplot as plt
from matplotlib import cm
%matplotlib inline #이미지를 노트북 안에 출력하도록 하는것

UsageError: unrecognized arguments: #이미지를 노트북 안에 출력하도록 하는것


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [22]:
train_raw = pd.read_csv('/content/drive/MyDrive/Dacon/태양광 발전량 예측 AI 경진대회/train/train.csv')
train= train_raw

In [23]:
submission = pd.read_csv('/content/drive/MyDrive/Dacon/태양광 발전량 예측 AI 경진대회/sample_submission.csv')

In [24]:
train.head()

,Day,Hour,Minute,DHI,DNI,WS,RH,T,TARGET
0,0,0,0,0,0,1.5,69.08,-12,0.0
1,0,0,30,0,0,1.5,69.06,-12,0.0
2,0,1,0,0,0,1.6,71.78,-12,0.0
3,0,1,30,0,0,1.6,71.75,-12,0.0
4,0,2,0,0,0,1.6,75.20,-12,0.0


In [10]:
train.iloc[:,8:]

,TARGET
0,0.0
1,0.0
2,0.0
3,0.0
4,0.0
...,...
52555,0.0
52556,0.0
52557,0.0
52558,0.0


In [26]:
def preprocess_data(data, is_train=True):
    
    temp = data.copy()
    temp = temp[['Hour', 'TARGET', 'DHI', 'DNI', 'WS', 'RH', 'T']]

    if is_train==True:          
    
        temp['Target1'] = temp['TARGET'].shift(-48).fillna(method='ffill')
        temp['Target2'] = temp['TARGET'].shift(-48*2).fillna(method='ffill')
        temp = temp.dropna()
        
        return temp.iloc[:-96] # 왜 마지막 2일을 빼고 학습?

    elif is_train==False:
       
        temp = temp[['Hour', 'TARGET', 'DHI', 'DNI', 'WS', 'RH', 'T']]
                              
        return temp.iloc[-48:, :] # 왜 마지막 날들만 빼서 데이터를 모으지?


df_train = preprocess_data(train)
df_train.iloc[:48]

,Hour,TARGET,DHI,DNI,WS,RH,T,Target1,Target2
0,0,0.000000,0,0,1.5,69.08,-12,0.000000,0.000000
1,0,0.000000,0,0,1.5,69.06,-12,0.000000,0.000000
2,1,0.000000,0,0,1.6,71.78,-12,0.000000,0.000000
3,1,0.000000,0,0,1.6,71.75,-12,0.000000,0.000000
4,2,0.000000,0,0,1.6,75.20,-12,0.000000,0.000000
5,2,0.000000,0,0,1.5,69.29,-11,0.000000,0.000000
6,3,0.000000,0,0,1.5,72.56,-11,0.000000,0.000000
7,3,0.000000,0,0,1.4,72.55,-11,0.000000,0.000000
8,4,0.000000,0,0,1.3,74.62,-11,0.000000,0.000000
9,4,0.000000,0,0,1.3,74.61,-11,0.000000,0.000000


In [28]:
df_test = []

for i in range(81):
    file_path = '/content/drive/MyDrive/Dacon/태양광 발전량 예측 AI 경진대회/test/' + str(i) + '.csv'
    temp = pd.read_csv(file_path)
    temp = preprocess_data(temp, is_train=False)
    df_test.append(temp)

X_test = pd.concat(df_test)
X_test.shape

(3888, 7)

In [29]:
X_test

,Hour,TARGET,DHI,DNI,WS,RH,T
288,0,0.0,0,0,0.8,80.92,-2.8
289,0,0.0,0,0,0.9,81.53,-2.9
290,1,0.0,0,0,1.0,79.91,-3.0
291,1,0.0,0,0,0.9,79.91,-3.0
292,2,0.0,0,0,0.9,77.20,-3.0
...,...,...,...,...,...,...,...
331,21,0.0,0,0,0.8,63.35,13.7
332,22,0.0,0,0,0.7,64.82,13.1
333,22,0.0,0,0,0.7,66.10,12.8
334,23,0.0,0,0,0.6,67.64,12.4


In [32]:
from sklearn.model_selection import train_test_split
X_train_1, X_valid_1, Y_train_1, Y_valid_1 = train_test_split(df_train.iloc[:, :-2], df_train.iloc[:, -2], test_size=0.25, random_state=0)
X_train_2, X_valid_2, Y_train_2, Y_valid_2 = train_test_split(df_train.iloc[:, :-2], df_train.iloc[:, -1], test_size=0.25, random_state=0)

In [11]:
#데이터 집합을 훈련 데이터와 테스트 데이터로 분할
train_X, test_X, train_Y, test_Y = model_selection.train_test_split(train.iloc[:,:-1], train.iloc[:,8:], train_size=0.7, test_size=0.3)

#이렇게하면 모든 

In [35]:
print(X_train_1)
print(X_valid_1)

       Hour     TARGET  DHI  DNI   WS     RH   T
42057     4   0.000000    0    0  1.1  95.70   4
457      12  49.359101   84  936  2.8  49.84   9
37100    22   0.000000    0    0  1.7  80.88   0
35024    16   9.384788   27  629  0.5  64.92   0
12571    21   0.000000    0    0  1.0  53.44  11
...     ...        ...  ...  ...  ...    ...  ..
45891     1   0.000000    0    0  1.8  84.63  13
52416     0   0.000000    0    0  3.4  50.47  -1
42613    18   9.664419   49  349  1.5  66.59  18
43567    15  65.014422  138  800  3.1  12.49  31
2732     22   0.000000    0    0  4.1  71.93 -12

[39348 rows x 7 columns]
       Hour     TARGET  DHI   DNI   WS     RH   T
47237     2   0.000000    0     0  1.1  80.65  14
49429    18   0.000000    0     0  0.5  53.20   0
27337    12  84.996275  330   615  3.4  23.19  32
28311    19   0.000000    0     0  0.7  50.55  17
5361     16  21.957266  185   116  0.6  45.08  13
...     ...        ...  ...   ...  ...    ...  ..
42888    12  99.453605   92  1009  4

In [37]:
#데이터를 array타입으로 변환

X_train_11 = np.array(X_train_1, dtype=float)
X_train_22 = np.array(X_train_2, dtype=float)

Y_train_11 = np.array(Y_train_1, dtype=float)
Y_train_22 = np.array(Y_train_2, dtype=float)

In [39]:
X_train_11.shape
#Y_train_11.shape

(39348, 7)

In [40]:
# train 데이터 3차원으로 만들어주기
X_train1 = X_train_11.reshape(1, 39348, 7)
X_train2 = X_train_22.reshape(1, 39348, 7)

Y_train1 = Y_train_11.reshape(1, 39348, 1)
Y_train2 = Y_train_22.reshape(1, 39348, 1)


In [41]:
#X_train1.shape
Y_train1.shape

(1, 39348, 1)

In [42]:
#test 데이터 array 타입으로 변환
X_test_11 = np.array(X_valid_1, dtype=float)
X_test_22 = np.array(X_valid_2, dtype=float)

Y_test_11 = np.array(Y_valid_1, dtype=float)
Y_test_22 = np.array(Y_valid_2, dtype=float)

In [43]:
X_test_11.shape
X_test_22.shape

Y_test_11.shape

(13116,)

In [44]:
# test 데이터 3차원으로 만들어주기
X_test1 = X_test_11.reshape(1, 13116, 7)
X_test2 = X_test_22.reshape(1, 13116, 7)

Y_test1 = Y_test_11.reshape(1, 13116, 1)
Y_test2 = Y_test_22.reshape(1, 13116, 1)

In [ ]:
train_X.shape
train_x = train_X.reshape(1,36724, 7)

In [ ]:
train_Y.shape
train_y = train_Y.reshape(1,36724, 1)

In [45]:
#훈련 데이터를 텐서 변환

train_X = torch.from_numpy(X_train_11).float()
train_Y = torch.from_numpy(Y_train_11).long()

In [46]:
#테스트 데이터 텐서 변환
test_X = torch.from_numpy(X_test_11).float()
test_Y = torch.from_numpy(Y_test_11).long()

In [47]:
#변환된 텐서의 데이터 건수 확인
print(train_X.shape)
print(train_Y.shape)

torch.Size([39348, 7])
torch.Size([39348])


##pytorch quantileloss


In [ ]:
class QuantileLoss(nn.Module):
    def __init__(self, quantiles):
        super(QuantileLoss, self).__init__()
        self.quantiles = quantiles
        
    def forward(self, preds, target):
        assert not target.requires_grad
        assert preds.size(0) == target.size(0)
        losses = []
        for i, q in enumerate(self.quantiles):
            errors = target - preds[:, i]
            losses.append(
                torch.max(
                   (q-1) * errors, 
                   q * errors
            ).unsqueeze(1))
        loss = torch.mean(
            torch.sum(torch.cat(losses, dim=1), dim=1))
        return loss

model=QuantileLoss(quantiles= [0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9])

## 딥러닝 다층퍼셉트론

In [ ]:
# 신경망 구성

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(7,7)
        self.fc2 = nn.Linear(7,7)
        self.fc3 = nn.Linear(7,48)
        
    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return F.log_softmax(x)
        
# 인스턴스 생성
model=Net()

In [ ]:
loss_func = nn.L1Loss()

optimizer = optim.SGD(model.parameters(), lr=0.01)

loss_array=[]

for epoch in range(1000):
  total_loss = 0
  optimizer.zero_grad()
  output = model(X_train_11)
  loss = loss_func(output, Y_train_11)
  loss.backward()
  optimizer.step()

  total_loss += loss.data
  loss_array.append(loss)

if (epoch+1) % 100 == 0:
        print(epoch +1, total_loss)  

AttributeError: ignored

In [ ]:
#pytorch 라이브러리 임포트
import torch
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

#scikit-learn 라이브러리
from sklearn.datasets import load_digits
from sklearn import datasets, model_selection

#pandas 라이브러리 임포트
import pandas as pd

#matplotlib 라이브러리 임포트
from matplotlib import pyplot as plt
from matplotlib import cm
%matplotlib inline #이미지를 노트북 안에 출력하도록 하는것

import numpy as np

UsageError: unrecognized arguments: #이미지를 노트북 안에 출력하도록 하는것


In [ ]:
data = np.array(train, dtype='float32')

data

array([[ 0.000e+00,  0.000e+00,  0.000e+00, ...,  6.908e+01, -1.200e+01,
         0.000e+00],
       [ 0.000e+00,  0.000e+00,  3.000e+01, ...,  6.906e+01, -1.200e+01,
         0.000e+00],
       [ 0.000e+00,  1.000e+00,  0.000e+00, ...,  7.178e+01, -1.200e+01,
         0.000e+00],
       ...,
       [ 1.094e+03,  2.200e+01,  3.000e+01, ...,  6.678e+01, -4.000e+00,
         0.000e+00],
       [ 1.094e+03,  2.300e+01,  0.000e+00, ...,  6.772e+01, -4.000e+00,
         0.000e+00],
       [ 1.094e+03,  2.300e+01,  3.000e+01, ...,  6.770e+01, -4.000e+00,
         0.000e+00]], dtype=float32)

In [ ]:
#input_size = 7
#hidden_size = 3

class RNN(nn.Module):
    
    # (batch_size, n, ) torch already know, you don't need to let torch know
    def __init__(self,input_size, hidden_size):
        super().__init__()
        self.input_size = input_size
        self.hidden_size = hidden_size
        
        self.rnn = nn.LSTM(
            input_size = input_size, 
            hidden_size = hidden_size, 
            num_layers = 4, 
            batch_first = True,
            bidirectional = True
        )
        
        self.layers = nn.Sequential(
            nn.ReLU(),
            nn.Linear(input_size*2, hidden_size),
        )
        
    def forward(self, x):
        y,_ = self.rnn(x)
        y = self.layers(y)
        return y
    
model = RNN(input_size=7, hidden_size=20)
model

RNN(
  (rnn): LSTM(7, 20, num_layers=4, batch_first=True, bidirectional=True)
  (layers): Sequential(
    (0): ReLU()
    (1): Linear(in_features=14, out_features=20, bias=True)
  )
)

#LSTM

In [48]:
class LSTM(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(LSTM, self).__init__()
        self.hidden_size = hidden_size
        self.lstm = nn.LSTM(input_size = input_size,
                            hidden_size = hidden_size,
                            batch_first=True)
        
        self.hidden_lstm = nn.LSTM(input_size = input_size,
                                   hidden_size = hidden_size,
                                   batch_first=True)
        
        self.time_fc = nn.Linear(hidden_size, 4)
    
    def forward(self, x_time):
    
        out_time, _ = self.lstm(x_time)
        out_time, _ = self.hidden_lstm(out_time)
        out_time, _ = self.hidden_lstm(out_time)
        out_time, _ = self.hidden_lstm(out_time)
        
        
        out_time = self.time_fc(out_time[:,-7:, :])
        
        return out_time.view(-1,7,4)
    
model = LSTM(input_size = 7, hidden_size = 40)
model

LSTM(
  (lstm): LSTM(7, 40, batch_first=True)
  (hidden_lstm): LSTM(7, 40, batch_first=True)
  (time_fc): Linear(in_features=40, out_features=4, bias=True)
)

In [ ]:
train_X.shape

torch.Size([36724, 7])

In [49]:
#X_train_11 = torch.tensor(X_train_11).float()
#Y_train_11 = torch.tensor(Y_train_11).float()

# Train model
optimizer = torch.optim.Adam(model.parameters(), lr = 1e-2)
criterion = nn.MSELoss(size_average = True)
num_epochs  = 500
train_error = []
for t in range(num_epochs):
    train_pred = model(train_X)
    loss = criterion(train_pred, train_Y) ### trend
    train_error.append(loss)
    
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    if t % 10 == 0 and t !=0:
        print(f"{t} Epochs train MSE: {loss.item():1.5f}")

RuntimeError: ignored

#혼자

In [ ]:
train.describe

<bound method NDFrame.describe of         Day  Hour  Minute  DHI  DNI   WS     RH   T  TARGET
0         0     0       0    0    0  1.5  69.08 -12     0.0
1         0     0      30    0    0  1.5  69.06 -12     0.0
2         0     1       0    0    0  1.6  71.78 -12     0.0
3         0     1      30    0    0  1.6  71.75 -12     0.0
4         0     2       0    0    0  1.6  75.20 -12     0.0
...     ...   ...     ...  ...  ...  ...    ...  ..     ...
52555  1094    21      30    0    0  2.4  70.70  -4     0.0
52556  1094    22       0    0    0  2.4  66.79  -4     0.0
52557  1094    22      30    0    0  2.2  66.78  -4     0.0
52558  1094    23       0    0    0  2.1  67.72  -4     0.0
52559  1094    23      30    0    0  2.1  67.70  -4     0.0

[52560 rows x 9 columns]>

In [ ]:
train.data = train.drop('TARGET', axis=1)
train.data

,Day,Hour,Minute,DHI,DNI,WS,RH,T
0,0,0,0,0,0,1.5,69.08,-12
1,0,0,30,0,0,1.5,69.06,-12
2,0,1,0,0,0,1.6,71.78,-12
3,0,1,30,0,0,1.6,71.75,-12
4,0,2,0,0,0,1.6,75.20,-12
...,...,...,...,...,...,...,...,...
52555,1094,21,30,0,0,2.4,70.70,-4
52556,1094,22,0,0,0,2.4,66.79,-4
52557,1094,22,30,0,0,2.2,66.78,-4
52558,1094,23,0,0,0,2.1,67.72,-4


In [ ]:
train.target=train.iloc[:,8]
train.target

0        0.0
1        0.0
2        0.0
3        0.0
4        0.0
        ... 
52555    0.0
52556    0.0
52557    0.0
52558    0.0
52559    0.0
Name: TARGET, Length: 52560, dtype: float64

In [ ]:
from sklearn.model_selection import train_test_split
train_X, test_X, train_Y, test_Y = train_test_split(train.data, train.target, test_size=0.3)

In [ ]:
print(len(train_X))
print(len(test_X))

36792
15768


In [ ]:
print(len(train_Y))
print(len(test_Y))

36792
15768


In [ ]:
#데이터를 array타입으로 변환

train_X = np.array(train_X, dtype=int)
train_Y = np.array(train_Y, dtype=int)

test_X = np.array(test_X, dtype=int)
test_Y = np.array(test_Y, dtype=int)

In [ ]:
# 훈련데이터 텐서 변환
train_X = torch.from_numpy(train_X).float()
train_Y = torch.from_numpy(train_Y).long()

# 테스트 데이터 텐서 변환
test_X = torch.from_numpy(test_X).float()
test_Y = torch.from_numpy(test_Y).long()

#텐서로 변환한 데이터 건수 확인
print(train_X.shape)
print(train_Y.shape)

torch.Size([36792, 8])
torch.Size([36792])


In [ ]:
#텐서로 변환한 데이터 건수 확인
print(train_X.shape)
print(train_Y.shape)

torch.Size([36792, 8])
torch.Size([36792])


In [ ]:
# 설명변수와 목적변수의 텐서를 합침
train=TensorDataset(train_X, train_Y)

#텐서의 첫 번째 데이터 내용 확인
print(train[0])

#미니 배치로 분할
train_loader = DataLoader(train, batch_size=4, shuffle=True)

(tensor([224.,   9.,   0.,  94., 848.,   1.,  36.,  28.]), tensor(62))


In [ ]:
# 신경망 구성

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(8,96)
        self.fc2 = nn.Linear(96,96)
        self.fc3 = nn.Linear(96,96)
        self.fc4 = nn.Linear(96,96)
        self.fc5 = nn.Linear(96,96)
        self.fc6 = nn.Linear(96,96)
        
    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = F.relu(self.fc4(x))
        x = F.relu(self.fc5(x))
        x = self.fc6(x)
        return F.log_softmax(x)
        
# 인스턴스 생성
model=Net()

In [ ]:
loss_func = nn.L1Loss()

optimizer = optim.SGD(model.parameters(), lr=0.01)

loss_array=[]

for epoch in range(1000):
  for train_x, train_y in train_loader:
        
        #계산 그래프 구성
        train_x, train_y = Variable(train_x), Variable(train_y)
        total_loss = 0
        optimizer.zero_grad()
        output = model(train_X)
        loss = loss_func(output, train_Y)
        loss.backward()
        optimizer.step()

        total_loss += loss.data
  loss_array.append(loss)

if (epoch+1) % 100 == 0:
        print(epoch +1, total_loss)  

RuntimeError: ignored